# 8760 Load Analysis 
## Purpose
The goal of this code is to take 8760 .csv files from electrical system and analyze when and where the load is not being met. It was specifically designed with the output from the HOMER microgrid software in mind. 

In [1]:
# import modules 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os 
import pathlib
from glob  import glob

In [2]:
#load csv file file path 
dir_path = os.path.join(
    pathlib.Path.home(),
        'code-projects',
        'load-8760'
        )


data_path = glob(os.path.join(dir_path,'*.csv'))[0]

data_path

'C:\\Users\\Nolan Welsh\\code-projects\\load-8760\\5pctCapShort.csv'

In [3]:
#load data and check structure 
data = pd.read_csv(
    data_path,
    delimiter=',',
    header= 0,
    index_col='Time',
    skiprows=1
    )

data = data[data.index.notnull()]

data.head()

,Global Solar,Generic flat plate PV Solar Altitude,Generic flat plate PV Solar Azimuth,Generic flat plate PV Angle of Incidence,Generic flat plate PV Incident Solar,Generic flat plate PV Power Output,AC Primary Load,AC Primary Load Served,Total Electrical Load Served,Renewable Penetration,...,Generic 1kWh Li-Ion Charge Power,Generic 1kWh Li-Ion Discharge Power,Generic 1kWh Li-Ion Input Power,Generic 1kWh Li-Ion Energy Content,Generic 1kWh Li-Ion State of Charge,Generic 1kWh Li-Ion Energy Cost,AC Required Operating Capacity,DC Required Operating Capacity,AC Operating Capacity,DC Operating Capacity
Time,,,,,,,,,,,,,,,,,,,,,
1/1/2007 12:00:00 AM,0,0,-54.52112,90,0,0,4.575846,4.575846,4.575846,0,...,0,4.81668,-4.81668,146.9231,96.65973,0,4.575846,0,0,115.3601
1/1/2007 1:00:00 AM,0,0,-73.37096,90,0,0,3.733734,3.733734,3.733734,0,...,0,3.930246,-3.930246,142.7802,93.93417,0,3.733734,0,0,110.5434
1/1/2007 2:00:00 AM,0,0,-75.68661,90,0,0,4.586835,4.586835,4.586835,0,...,0,4.828248,-4.828248,137.6908,90.58588,0,4.586835,0,0,106.6132
1/1/2007 3:00:00 AM,0,0,-74.9487,90,0,0,5.179585,5.179585,5.179585,0,...,0,5.452195,-5.452195,131.9437,86.80489,0,5.179585,0,0,101.7849
1/1/2007 4:00:00 AM,0,0,-72.89027,90,0,0,5.075244,5.075244,5.075244,0,...,0,5.342362,-5.342362,126.3124,83.10007,0,5.075244,0,0,96.33275


Separate time into hour, day, and month

In [4]:
# Convert time to hour, day, and month

#Reset Index 
data=data.reset_index()


data["Time"] = pd.to_datetime(data["Time"])

data["hour"] = data['Time'].map(lambda x: x.hour)
data["day"] = data['Time'].map(lambda x: x.day)
data["month"] = data['Time'].map(lambda x: x.month)

data.head()



C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_5244\3181407025.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Time"] = pd.to_datetime(data["Time"])


,Time,Global Solar,Generic flat plate PV Solar Altitude,Generic flat plate PV Solar Azimuth,Generic flat plate PV Angle of Incidence,Generic flat plate PV Incident Solar,Generic flat plate PV Power Output,AC Primary Load,AC Primary Load Served,Total Electrical Load Served,...,Generic 1kWh Li-Ion Energy Content,Generic 1kWh Li-Ion State of Charge,Generic 1kWh Li-Ion Energy Cost,AC Required Operating Capacity,DC Required Operating Capacity,AC Operating Capacity,DC Operating Capacity,hour,day,month
0,2007-01-01 00:00:00,0,0,-54.52112,90,0,0,4.575846,4.575846,4.575846,...,146.9231,96.65973,0,4.575846,0,0,115.3601,0,1,1
1,2007-01-01 01:00:00,0,0,-73.37096,90,0,0,3.733734,3.733734,3.733734,...,142.7802,93.93417,0,3.733734,0,0,110.5434,1,1,1
2,2007-01-01 02:00:00,0,0,-75.68661,90,0,0,4.586835,4.586835,4.586835,...,137.6908,90.58588,0,4.586835,0,0,106.6132,2,1,1
3,2007-01-01 03:00:00,0,0,-74.9487,90,0,0,5.179585,5.179585,5.179585,...,131.9437,86.80489,0,5.179585,0,0,101.7849,3,1,1
4,2007-01-01 04:00:00,0,0,-72.89027,90,0,0,5.075244,5.075244,5.075244,...,126.3124,83.10007,0,5.075244,0,0,96.33275,4,1,1


# Calculation 
Since the data is loaded, we are going to filer the amount of hours were capacity shortage > 0 

In [5]:
# filter  how many hours load not met (capacity shortage > 0)
#first need to force column to convert to float 
data['Capacity Shortage'] = data['Capacity Shortage'].astype(float)

capacity_shortage_df = data[data['Capacity Shortage'] > 0 ]

#Calculate how man hour load is unmet   

hours_shortage = len(capacity_shortage_df)

hours_shortage

721

Now to try with battery capacity (<30%)



In [6]:
data['Generic 1kWh Li-Ion State of Charge'] = data['Generic 1kWh Li-Ion State of Charge'].astype(float)

battery_shortage_df = data[data['Generic 1kWh Li-Ion State of Charge'] < 30 ]

hours_battery_shortage = len(battery_shortage_df)

hours_battery_shortage


789

In [ ]:
#hours with unmet electrical load 
data['Unmet Electrical Load'] = data['Unmet Electrical Load'].astype(float)

unmet_load_df = data[data['Unmet Electrical Load'] > 0.01 ]

hours_unmet_load = len(unmet_load_df)

hours_per_day = unmet_load_df.groupby(['month','day']).size()

num_days_outages = len(hours_per_day)

hours_per_day

month  day
1      1      10
       2       7
       4       7
       5      14
       6       7
              ..
12     6       7
       19      6
       22     11
       23      6
       25      3
Length: 72, dtype: int64

In [ ]:
# find consecutive hours with outages 

def count_consecutive_groups(df):
    groups = (df['hour'].diff() != 1).cumsum()  # Identify consecutive groups
    return len(groups.unique()) 

outages_year = count_consecutive_groups (unmet_load_df)

outages_year

 

101

In [9]:
outages_day = unmet_load_df.groupby(['month', 'day']).apply(count_consecutive_groups)

outages_day

C:\Users\Nolan Welsh\AppData\Local\Temp\ipykernel_5244\2700330661.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outages_day = unmet_load_df.groupby(['month', 'day']).apply(count_consecutive_groups)


month  day
1      1      1
       2      1
       4      1
       5      2
       6      1
             ..
12     6      1
       19     1
       22     1
       23     1
       25     1
Length: 72, dtype: int64

Print all  results 

In [ ]:
pd.set_option('display.max_rows', None)
print(hours_per_day)
print(outages_day)
print(
    'Hours with Capacity Shortage=',
    hours_shortage, 'hours')

print(
    'Hours with Battery Capacity <30%= ',
    hours_battery_shortage, 'hours'
)
print(
    '# of Days with Outages (unmet load)= ',
    num_days, 'days'
)
print(
    '# of outages per year= ',
    outages_year, 'outages'
)


month  day
1      1      10
       2       7
       4       7
       5      14
       6       7
       13      3
2      3       4
       8      12
       9       9
       10      7
3      24      2
       27     12
       28      6
4      8      10
       9       6
       11      3
       29      1
       30      6
5      8      13
       9       6
       13      9
       14      6
       17     11
       18     15
       19      6
6      17     14
       18     13
       19      6
       25      4
       26     18
       27      6
7      21      1
       22      6
8      2      19
       3      20
       4       6
9      2       8
       3      13
       4       6
       15     10
       16      6
       17      6
       27     13
       28      6
       29      8
       30      6
10     11     11
       12      6
       18      1
       29      4
       30      7
       31     10
11     1       6
       8      13
       9      14
       10      6
       18      9
       19      6
   